In [1]:
import os
from athina.evals import (
    RagasContextRelevancy,
    RagasAnswerRelevancy,
    RagasFaithfulness,
    RagasAnswerCorrectness,
)
from athina.runner.run import EvalRunner
from athina.loaders import RagasLoader
from athina.keys import AthinaApiKey, OpenAiApiKey
from llama_index import VectorStoreIndex, ServiceContext
from llama_index import download_loader
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

/Users/shivsakhuja/dev/athina/athina-evals/.venvtest/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# create a llamaindex query engine
WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
documents = loader.load_data(pages=['Y Combinator'])
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)

query_engine = vector_index.as_query_engine()

In [3]:
raw_data_llama_index = [
    {
        "query": "How much equity does YC take?",
        "expected_response": "YC invests $500k in exchange for 7 percent equity."
    },
    {
        "query": "Who founded YC?",
        "expected_response": "YC was founded by Paul Graham, Jessica Livingston, Robert Tappan Morris, and Trevor Blackwell."
    },
]

In [4]:
llama_index_dataset = RagasLoader(query_engine=query_engine).load_dict(raw_data_llama_index)
pd.DataFrame(llama_index_dataset)

,query,contexts,response,expected_response
0,How much equity does YC take?,[Tan's leadership marked the discontinuation o...,YC takes a 7% equity stake in companies that p...,YC invests $500k in exchange for 7 percent equ...
1,Who founded YC?,"[Y Combinator Management, LLC (YC) is an Ameri...","Paul Graham, Jessica Livingston, Robert Tappan...","YC was founded by Paul Graham, Jessica Livings..."


In [5]:
eval_model = "gpt-3.5-turbo"
eval_suite = [
    RagasAnswerCorrectness(),
    RagasFaithfulness(),
    RagasContextRelevancy(),
    RagasAnswerRelevancy(),
]

# Run the evaluation suite
EvalRunner.run_suite(
    evals=eval_suite,
    data=llama_index_dataset,
    max_parallel_evals=1,   # If you increase this, you may run into rate limits
)

Running RagasAnswerCorrectness evaluator
evaluating with [answer_correctness]


100%|██████████| 1/1 [00:03<00:00,  3.90s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [00:04<00:00,  4.03s/it]


Running RagasFaithfulness evaluator
evaluating with [faithfulness]


...

100%|██████████| 1/1 [00:02<00:00,  2.60s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:02<00:00,  2.29s/it]


You can view the evaluation results at https://app.athina.ai/develop/request/b2b36246-de26-4a98-b867-c0c6e272ef09


,query,contexts,response,expected_response,ragas_answer_correctness,ragas_faithfulness,ragas_context_relevancy,ragas_answer_relevancy
0,How much equity does YC take?,[Tan's leadership marked the discontinuation o...,YC takes a 7% equity stake in companies that p...,YC invests $500k in exchange for 7 percent equ...,0.722709,1.0,0.285714,0.927897
1,Who founded YC?,"[Y Combinator Management, LLC (YC) is an Ameri...","Paul Graham, Jessica Livingston, Robert Tappan...","YC was founded by Paul Graham, Jessica Livings...",0.985727,1.0,0.029412,0.937349
